In [ ]:
#https://motius.breezy.hr/p/0b622aa1163e01-join-our-tech-community-as-working-student-m-f-d/apply?utm_source=pt-linkedin&utm_medium=job-posting

In [ ]:
"""!pip install datasets
!pip install transformers
!pip install sentence_transformers
!pip install peft
!pip install huggingface_hub
!pip install --upgrade accelerate
!pip install --upgrade bitsandbytes
!pip install --upgrade torchmetrics
!pip install --upgrade deepspeed
"""

In [ ]:
from transformers import  AutoModel,AutoTokenizer,AutoModelForMaskedLM , Trainer,TrainingArguments,\
BitsAndBytesConfig,pipeline,default_data_collator,DataCollatorWithPadding,DataCollatorForLanguageModeling
from transformers.utils import move_cache
#from chat_template_utils import get_json_schema
#from llama_cpp import Llama
#from utils import *
from peft import *
import datasets
#import torchmetrics
import torch
from torch.utils.data.dataloader import DataLoader,Dataset
#from hqq.engine.hf import HQQModelForCausalLM
#from hqq.models.hf.base import AutoHQQHFModel
from huggingface_hub import snapshot_download
#import deepspeed
import os
import pandas as pd
import json
#from var_dump import var_dump
from bs4 import BeautifulSoup,PageElement,Comment
import matplotlib.pyplot as plt
import seaborn as sns

cache_dir='/var/tmp/.cache/' #'/proj/ciptmp/ix05ogym/.cache/'
output_dir = cache_dir+'outputs/'

move_cache(cache_dir)


In [ ]:
train_dataset = pd.read_excel('train_forms.xlsx')[['input_text','labels']]
train_dataset.drop_duplicates(['input_text']).reset_index(drop=True)
c = train_dataset['labels']=='other'
train_dataset.loc[c,'labels']=pd.NA
c = train_dataset['labels']=='birthdate'
train_dataset.loc[c,'labels']='birth'

train_dataset['labels'].unique()

#c = train_dataset['labels']=='privacy'
#train_dataset.loc[c,:]

lll=[]
person = {"first name":"majid","last name":"sharghi foroushani","pronoun":"he","birth":"10/08/1997","age":"27","sex":"male","skill":"NLP,LLM","country":"Germany","address":"Philip-9","city":"Erlangen","phone number":"+497773154","available from":"6/24/2024","salary":"10000","German language level":"A1","English language level":"C1","visa":"do not need visa","email":"sharghi.majid@gmail.com","password":"123456","find us":"Google","xing":"www.xing.com","job":"I am stduent","university":"FAU","major":"AI","privacy":"privacy","search":"search","submit":"submit","cookie":"cookie","cover letter":"cover letter","linkedin":"www.linkedin.com","github":"www.github.com","login":"login","register":"register","cv":"/pah/to/cv.pdf","twitter":"www.twitter.com"}
for k,v in person.items():
    c =  train_dataset['labels']==k
    #train_dataset.loc[c,'labels']=k
    pass

print(train_dataset['labels'].value_counts())


In [ ]:
test_dataset = pd.read_excel('forms.xlsx')
c = test_dataset['label'].notna()
test_dataset = test_dataset.loc[c,:].reset_index(drop=True)
test_dataset['label'].unique()

In [ ]:
from transformers import Qwen2ForCausalLM
device = 'cuda'
tokenizer = AutoTokenizer.from_pretrained('Qwen/Qwen2-0.5B-Instruct',cache_dir=cache_dir)
model = Qwen2ForCausalLM.from_pretrained('Qwen/Qwen2-0.5B-Instruct' #Qwen/Qwen2-0.5B-Instruct
                                         #,attn_implementation="flash_attention_2"
                                         #,torch_dtype=torch.float16
                                         , device_map='auto',cache_dir=cache_dir).to('cuda')

model.eval()
#model.config['use_sliding_window']=True
print(model.config)

model

In [ ]:
model.generation_config

In [ ]:
model = get_peft_model(model,LoraConfig(task_type=TaskType.CAUSAL_LM,target_modules=['q_proj','v_proj'
                                                                                     #,'k_proj','o_proj'
                                                                                     ]))
model.print_trainable_parameters()
model

In [ ]:

c = test_dataset['label']=='guthub'
test_dataset.loc[c,'label'] = 'github'
test_dataset = test_dataset.reset_index(drop=True)

In [ ]:
#test_dataset.to_excel('forms.xlsx',index=0)

In [ ]:
x = test_dataset['query'] + test_dataset['element']
x = x.tolist()
x[0]

In [ ]:
KEYWORDS={
        #----------------------form--------------------------------------
       'first name':['firstname','first_name','forename','first','first-name'],
       'last name':['lastname','last_name','surename','last','last-name'] ,
       'email':['email','e-mail','username','user_name','user-name','type="email"'],
       'password':['password'],
       'cv':['cv','resume','curriculum','vitae'],
       'birth date':['birthdate','birth','dateofbirth','date-of-birth','birth-date'],
       'phone number':['phonenumber','phone','telephone','tel'],
       'house number':['hausnummer'],
       'resident country':['resident','country'],
       'citizen country':['citizen','citizenship','city'],
       'transcript':['transcript'],
       'cover letter':['coverletter','cover', 'letter'],
       'picture':['picture'],
       'address':['address','location'],
       'linkedin':['linkedin'],
       'github':['github'],
       'xing':['xing'],
       'twitter':['twitter'],
       'website':['website'],
       'zip':['postalcode','zip','zip-code','postal'],
       'city':['city'],
       'salary':['salary'],
       'sex':['sex','gender'],
       'available from':['noticeprieod'],
       #'file':['type="file"'],

       'captcha':['captcha'],
       'submit':['apply','type="submit"'],
       'cancel':['cancel'],
       'fake':['fake'],
       'custom question':['question'],
       'privacy':['agree'],

       'recommend':['recommendation'],
       'work here before':['workherebefore'],
       'find us':['find'],
       'dropbox':['dropbox'],
       'google drive':['google drive'],
       'work hours':['workhours'],
       'workduration':[''],
       'visa':['visa'],
       #'eligible':['eligible'],
       'german language level':['germanlevel'],
       'company':[''],#current company
       'willing to relocate':[''],
       'university enrollment':[],#are a student?
       'english language level':[],
       #important about job
       #important about jobplace?

       #-------------------------------------------------------------------
       #-----------------------------------btns ---------------------------
       'login':['login','sign'],
       'username':['username'],
       'register':['register','sign'],
       'cookie':['cookie'],
       'next':['apply now','next'],
       'search':[],
        'other':['other'],
        'newsletter':[]
       #'apply':['apply'],
       #-------------------------------------------------------------------

       }

LABEL_INDEX_TO_KEY = list(KEYWORDS.keys())

i =0
LABEL_KEY_TO_INDEX = {}
for key in KEYWORDS:
            LABEL_KEY_TO_INDEX[key] =i
            i+=1

In [ ]:
s='['
for i in LABEL_INDEX_TO_KEY:
    s+=f'"{i}",'
    pass

s+=']'
s
#str(LABEL_INDEX_TO_KEY[0])

In [ ]:
"""def f(x):
    print(x)
    return '{"answer":"'+ x +'"}'

train_dataset['labels'] = train_dataset['labels'].map(f)
train_dataset"""


In [ ]:
import random
def get_random():
    d = {}
    for k in person.keys():
        d[k]=str(random.randint(1000,10000))

    return d

#get_random()
def to_str(p):
    s='{'
    for k,v in p.items():
        s+=f'"{k}":"{v}",'
        pass

    s+='}'
    return s


def get_one_msg(text , label):
    p = get_random()
    strp = to_str(p)
    rc = ''#random.choice(list(p.keys()))+' '
    d = [#{"role":"system","content":""}
    { "role":"user", "content":"""Your answer has to be in json format like {"answer":""}. Use this content in your answer:\n"""+strp}
    ,{"role":"user", "content":"What is this HTML element about?\n"+rc+text }
    ,{"role":"assistant","content":'{"answer":"'+ p[label] +'"}'}
        ]
    return d
def get_chat():

    d = []
    labels = []
    for i,x in train_dataset.iterrows():
        try:
                d.append(get_one_msg(x['input_text'],x['labels']))
        except:
            #print(x['labels'])
            pass

    return d#,labels

get_chat()

In [ ]:
test_dataset

In [ ]:
def get_train_data():
    return tokenizer.apply_chat_template(get_chat(),add_generation_prompt=False ,truncation=True,max_length=512)  # ,return_tensors='pt')
    pass

tokenized_data = get_train_data()


no_label = tokenizer.apply_chat_template([ [#{"role":"system","content":""}
                                                {"role":"user", "content":"""Your answer has to be in json format like {"answer":""}. Use this content in your answer:\n"""+to_str(person)}
                               ,{"role":"user", "content":"What is this HTML element about?\n"+str(x['query'])+x['element'] }
                               #,{"role":"assistant","content":""}
                               ] for i,x in test_dataset.iterrows()] ,truncation=True,max_length=512)  # ,return_tensors='pt')

print(len(tokenized_data))

print(tokenizer.decode(token_ids=tokenized_data[0]))
print(tokenizer.decode(token_ids=no_label[0]))


In [ ]:
tokenizer(['<|im_start|>assistant'])

In [ ]:
mydc = DataCollatorForLanguageModeling(tokenizer,False)
def mmf(x):
    y = mydc(x)
    b = (y['labels']==77091).int().argmax()-1
    for i in range(0,b):
        y['labels'][0][i]=-100
        pass
    #print( y['labels'] )
    return y

mmf(no_label)['labels'].shape


In [ ]:
from transformers import TrainerCallback
"""
class ChangeDatasetCallback(TrainerCallback):
    def __init__(self):
        #super.__init__()
        pass

    def on_epoch_end(self, args, state, control, **kwargs):
        # Move to the next dataset at the end of each epoch
        #self.current_epoch += 1
        #new_dataset = self.datasets[self.current_epoch % len(self.datasets)]
        kwargs['train_dataloader'].dataset = get_train_data()
        #kwargs['eval_dataloader'].dataset = new_dataset['validation']

"""

class custom_dataset(Dataset):
    def __init__(self) -> None:
        super().__init__()
        self.dataset = get_train_data()
        self.len = len(self.dataset)
        pass

    def __getitem__(self, i) :
        if i==self.len-1:
            self.dataset = get_train_data()

        return self.dataset[i]
        pass

    def __len__(self):
        return self.len
        pass





trainer =  Trainer(model,
        train_dataset=custom_dataset(),
        data_collator=mmf,
        #compute_metrics=compute_metric,
        #callbacks=[ChangeDatasetCallback()],
        args=TrainingArguments(
          output_dir=output_dir,
          num_train_epochs=8,
          per_device_train_batch_size=1,
          per_device_eval_batch_size=1,
          logging_steps=300,

          #eval_accumulation_steps =2,


        ))

trainer.train()

In [ ]:
def compute_metric():
    model.eval()
    acc=0
    with torch.no_grad():
        for i in range(len(no_label)):
            x = mydc([no_label[i]])['input_ids'].to('cuda')
            #print(x)
            o= model.generate(input_ids=x,max_new_tokens=20,do_sample=False)
            decoded = tokenizer.decode(o[0,x.shape[1]:])
            #print(decoded)
            #break

            #print(x[0],x[1])
            try:
                _s = decoded.find('{')
                _e = decoded.find('}',_s)+1
                #print(decoded[_s:_e])
                answer = json.loads(decoded[_s:_e])['answer']
                c = answer==person[test_dataset.loc[i,'label']]

                if c:
                    acc +=1
                else:
                    print("NO_CORRECT:",answer,person[test_dataset.loc[i,'label']])
                    print("---------------------------------")

                    pass


                pass
            except Exception as e:
                print("ERROR: ",decoded)
                print("---------------------------------")

                print(e)

                pass

    acc/=len(no_label)
    print("acc:",acc)



compute_metric()

In [ ]:
model.eval()
i =11
d = mydc(no_label[i:i+1])['input_ids'].to(device)
d = tokenizer.apply_chat_template( [#{"role":"system","content":""}
                             {"role":"user", "content":"""Your answer has to be in json format like {"answer":""}. Use this content in your answer:\n"""+s}
                            ,{"role":"user", "content":'email'}] , return_tensors='pt').to(device)
#d = tokenizer(['json {"answer":""} what is this html element about? <input type="email">'],return_tensors='pt')['input_ids'].to(device)
#print(d)
with torch.no_grad():

    o = model.generate(input_ids = d,max_length = 512,do_sample=True)
    print(tokenizer.decode(o[0]))



In [ ]:
$$$

LABEL_INDEX_TO_KEY

In [22]:
from sentence_transformers  import SentenceTransformer
#from transformers import AutoModel

embdeding_tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L12-v2',cache_dir=cache_dir)

#embeding_model= AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L12-v2',cache_dir=cache_dir)#.to('cuda')
embeding_model= SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2',cache_folder=cache_dir)#.to('cuda')


embeding_model.eval()
embeding_model

/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [ ]:
data = pd.read_excel('query_element_3.xlsx')
data

In [ ]:
embeding_model = get_peft_model(embeding_model,LoraConfig(target_modules=['query','value']))
embeding_model.print_trainable_parameters()


In [ ]:

#c = train_dataset['labels']=='relocate'
#train_dataset.loc[c,'labels'] = 'willing to relocate'
#train_dataset[c]


In [ ]:
#train_dataset.to_excel('train_forms.xlsx',index=0)

In [ ]:
from sentence_transformers.trainer import SentenceTransformerTrainer,SentenceTransformerTrainingArguments
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


In [ ]:
data.loc[:,'label']=1
train_dataset = datasets.Dataset.from_pandas(data)
train_dataset

In [ ]:
def batch_tok(example):
    example['query']= embdeding_tokenizer(example['query'])
    example['element']= embdeding_tokenizer(example['element'])

train_dataset.map(batch_tok)

In [ ]:
def batch_tok(example):

    return embdeding_tokenizer(example,padding=True,truncation=True,max_length=512,return_tensors='pt')



LABEL_INDEX_TO_KEY = list(KEYWORDS.keys())

query_token = embdeding_tokenizer(data['query'].tolist(),padding=False,truncation=True,max_length=512)
element_token = embdeding_tokenizer(data['element'].tolist(),padding=False,truncation=True,max_length=512)
classes_token = embdeding_tokenizer(LABEL_INDEX_TO_KEY,padding=True,truncation=True,max_length=512,return_tensors='pt')


def myheatmap(x):
    map =[]
    for i in range(len(x)):
        sim = torch.cosine_similarity(x,x[i])
        map.append(sim)


    plt.figure(figsize=(10,10))
    sns.heatmap(map)
    plt.show()



with torch.no_grad():

    output = embeding_model(**classes_token)
    #class_embdeding = torch.mean(output.last_hidden_state,1)
    class_embdeding = mean_pooling(output,classes_token['attention_mask'])
    class_embdeding = torch.nn.functional.normalize(class_embdeding, p=2, dim=1)
    #sim = torch.cosine_similarity(class_embdeding,class_embdeding[0])

    #print(sim.shape)
    pass

i = 151
with torch.no_grad():
    x = torch.tensor(element_token['input_ids'][i:i+1])
    #print(x)
    output = embeding_model(input_ids= x)
    #query_embdeding = torch.mean(output.last_hidden_state,1)
    query_embdeding = mean_pooling(output,torch.tensor(element_token['attention_mask'][i:i+1]))
    #query_embdeding = torch.nn.functional.normalize(class_embdeding, p=2, dim=1)

    sim = torch.cosine_similarity(class_embdeding,query_embdeding)
    print(pd.DataFrame({'value':sim,'label':LABEL_INDEX_TO_KEY}).sort_values('value',ascending=False)[:5])
    print(LABEL_INDEX_TO_KEY[ sim.argmax() ], embdeding_tokenizer.decode(token_ids=x[0]))
    print(data['element'][i] )


#myheatmap(class_embdeding)


In [ ]:
embeding_model.tokenizer = embdeding_tokenizer
embeding_model.tokenize = embdeding_tokenizer.tokenize

from sentence_transformers import SentenceTransformerModelCardData
from sentence_transformers.losses import CosineSimilarityLoss
#embeding_model.model_card_data = SentenceTransformerModelCardData(train_datasets=train_dataset)

In [ ]:
trainer  =  SentenceTransformerTrainer(model=embeding_model,
                           train_dataset=train_dataset,
                           tokenizer=embdeding_tokenizer,
                           loss=CosineSimilarityLoss(embeding_model),
                           args=SentenceTransformerTrainingArguments(
                               output_dir=output_dir,
                               num_train_epochs=1,
                               per_device_train_batch_size=1
                           )

                           )


trainer.train()

In [ ]:
test_dataset

In [ ]:
def compute_metric():
    embeding_model.eval()
    acc=0

    with torch.no_grad():

        l=embeding_model.encode(LABEL_INDEX_TO_KEY)
        l = torch.tensor(l)
        print(l.shape)

        for i,x in test_dataset.iterrows():
            #te = embdeding_tokenizer([x['element']],max_length=512,truncation=True,return_tensors='pt')['input_ids']

            o= embeding_model.encode( x['element'])
            o = torch.tensor(o)
            sim = torch.cosine_similarity(l,o)
            #print(sim.shape)
            #sim = torch.mean(sim,1)
            if LABEL_INDEX_TO_KEY[sim.argmax()]==x['label']:
                acc+=1
                pass
            #print(LABEL_INDEX_TO_KEY[sim.argmax()],x['label'])




    acc/=len(test_dataset)
    print("acc:",acc)



compute_metric()

In [ ]:
$$$

In [ ]:
with open('./html/1.html','r',encoding='utf-8') as f:
    html = f.read()


html = BeautifulSoup(html,'html.parser')

In [ ]:
def preprocess_form(form):
    if isinstance(form,str):
        form = BeautifulSoup(form,'html.parser')

    form:PageElement
    comments=form.find_all(string=lambda text: isinstance(text, Comment))
    for comment in comments:
        comment.extract()
    all_tags = form.find_all()

    # Remove all tags except for 'link', 'input', and 'select' tags
    i=0
    for element in all_tags:
        element:PageElement
        for k in ['style','class','bounding_box_rect','is_clickable']:
            if k in element.attrs:
                    del element[k]
                    pass

        if 'type' in element.attrs and element.attrs['type']=='hidden':
            element.unwrap()
            pass


        #if element.name  in ['div','text']:
        if element.name not in ['a', 'input', 'select','radio','button','textarea','checkbox','option']:

            element.unwrap()
            #print(element)
            pass
        else:
            #element['backend_node_id']=i
            pass



            i+=1

    #print(form.prettify() )
    return form

import re

def get_query_element(f):
    query_element=[]
    mytext  = ''
    texts = []

    for element in f:
        # Get the text before the element
        #print('-----------------------------')
        if element.name == None:
            t = element.text.replace('\n','').replace('\t','').strip()
            if t!='' and len(t)>2:
                #mytext+=t+'\n '
                texts.append(t)
        else:
            mytext = '\n'.join(texts[-1:])
            query_element.append({'query':mytext,'element': str(element).replace('\n','').replace('  ','').strip()})
            mytext =''
            texts=[]

    return query_element


def get_query_html(f):
    f:PageElement
    childs = f.find_all(string=True,recursive=False)
    for c in childs:
        print('---------------------')
        print(c)
    pass



print(forms.loc[10,'form'])
f = preprocess_form(forms.loc[25,'form'])
d = get_query_element(f)
d = pd.DataFrame(d)
for i,x in d.iterrows():
    print(x['query'])
    print(x['element'])
    print('--------------------------------------------------')

In [ ]:
forms=pd.read_excel('forms copy.xlsx',index_col=0).reset_index(drop=True)

In [ ]:
for _ ,f in forms.iterrows():
    print(_)
    #print(f['form'])
    f = str(f['form'])
    #print(f)
    #f = BeautifulSoup(str(f),'html.parser')
    f = preprocess_form(f)
    #f = f.find_all()
    #print(f)
    d = get_query_element(f)
    d = pd.DataFrame(d)
    #print(d)
    for i,x in d.iterrows():
        print(x['query'])
        print(x['element'])
        print('--------------------------------------------------')


    print('***************************************************')
    input()

In [ ]:
qe = pd.read_excel('query_element_3.xlsx')
qe

In [ ]:
model.eval()
toks = tokenizer.apply_chat_template([ [#{"role":"system","content":""}
                                {"role":"user", "content":"""Your answer has to be in json format like {"answer":""}. Use this content in your answer:\n"""+to_str(person)}
                               ,{"role":"user", "content":"What is this HTML element about?\n"+str(x['query'])+x['element'] }
                               #,{"role":"assistant","content":""}
                               ] for i,x in qe.iterrows()] ,truncation=True,max_length=512)  # ,return_tensors='pt')





In [ ]:
from ast import keyword
import json
with torch.no_grad():
  for i in range(0,len(toks)):
      t = toks[i]
      t = torch.tensor([t]).cuda()

      o = model.generate(input_ids=t,max_new_tokens=20,do_sample=False,)
      decoded = tokenizer.decode(o[0][t.shape[1]:])
      #print(decoded)

      try:
                _s = decoded.find('{')
                _e = decoded.find('}',_s)+1
                #print(decoded[_s:])
                #print(decoded[_s:_e])
                answer = json.loads(decoded[_s:_e])['answer']
                print(i,answer)

                for k,v in person.items():
                    if v==answer:

                      qe.loc[i,'label'] = k
                    else:
                      qe.loc[i,'label'] = pd.NA

                qe.loc[i,'answer']=answer


                pass
      except Exception as e:
                print("ERROR: ",decoded)
                print("---------------------------------")

                print(e)

                pass


In [ ]:
def auto_label(x):
    element = BeautifulSoup(x['element'],'html.parser').find()
    #print(element)
    type = element.attrs.get('type')
    print(type)
    if type=='email':
        return 'email'
    elif type=='search':
        return 'search'
    elif type=='password':
        return 'password'
    elif type=='tel':
        return 'phone number'
    return pd.NA
    
    
value_to_key = {}
for k,v in person.items():
    value_to_key[v]=k
    
    
for i,x in qe.iterrows():
    #l  = auto_label(x)
    try:
        l = value_to_key[x['answer']]
        qe.loc[i,'label'] = l
        #print(l)
    except:
        qe.loc[i,'label'] = pd.NA
        pass
    #print(label)
    

In [33]:
person

{'first name': 'majid',
 'last name': 'sharghi foroushani',
 'pronoun': 'he',
 'birth': '10/08/1997',
 'age': '27',
 'sex': 'male',
 'skill': 'NLP,LLM',
 'country': 'Germany',
 'address': 'Philip-9',
 'city': 'Erlangen',
 'phone number': '+497773154',
 'available from': '6/24/2024',
 'salary': '10000',
 'German language level': 'A1',
 'English language level': 'C1',
 'visa': 'do not need visa',
 'email': 'sharghi.majid@gmail.com',
 'password': '123456',
 'find us': 'Google',
 'xing': 'www.xing.com',
 'job': 'I am stduent',
 'university': 'FAU',
 'major': 'AI',
 'privacy': 'privacy',
 'search': 'search',
 'submit': 'submit',
 'cookie': 'cookie',
 'cover letter': 'cover letter',
 'linkedin': 'www.linkedin.com',
 'github': 'www.github.com',
 'login': 'login',
 'register': 'register',
 'cv': '/pah/to/cv.pdf',
 'twitter': 'www.twitter.com'}

In [101]:
#qe['label'].value_counts()
c1 =qe['label'].isna()
#c2 = qe['auto_label']=='password' 
rows = qe[c1]

rows
#qe[qe['auto_label']=='email']

#qe[qe['auto_label'].notna()]

,query,element,answer,label
0,keywords,"<input autocomplete=""off"" autofocus="""" id=""aut...",NaN,<NA>
4,Subscribe,"<button type=""submit"">Subscribe</button>",subscribe,<NA>
7,Apply now,"<input id=""login_button_top"" name=""login_butto...",NaN,<NA>
8,Apply now,"<input id=""login_button_bottom"" name=""login_bu...",NaN,<NA>
9,Your Name,"<input aria-describedby=""ph-a11y-email-job-nam...",sharghi.majid,<NA>
...,...,...,...,...
1583,The first 6 months of working at Similarweb ar...,"<input aria-required=""true"" autocomplete=""cust...",NaN,<NA>
1585,Are you Hispanic/Latino? Please selectYes\nNo\...,"<select aria-required=""false"" id=""job_applicat...",Yes,<NA>
1588,Veteran Status Please selectI am not a protect...,"<select aria-required=""false"" id=""job_applicat...",I am not a protected veteran,<NA>
1589,"Completing this form is voluntary, and we hope...","<a href=""https://www.dol.gov/ofccp"" target=""_b...",https://www.dol.gov/ofccp,<NA>


In [98]:
qe

,query,element,answer,label
0,keywords,"<input autocomplete=""off"" autofocus="""" id=""aut...",NaN,<NA>
1,Email Address,"<input id=""fieldEmail"" name=""cm-ukutgu-ukutgu""...",sharghi.majid@gmail.com,email
2,Please tick to confirm that you consent to us ...,"<a href=""/privacy-policy"" target=""_blank"" titl...",privacy,privacy
3,which includes details on how you can withdraw...,"<input name=""gdpr_verification"" required="""" ty...",cookie,cookie
4,Subscribe,"<button type=""submit"">Subscribe</button>",subscribe,<NA>
...,...,...,...,...
1586,Please identify your race Please selectAmerica...,"<select aria-required=""false"" id=""job_applicat...",American Indian or Alaskan Native,ethnicity
1587,Race & Ethnicity Definitions,"<a href=""https://boards.cdn.greenhouse.io/docs...",A1,German language level
1588,Veteran Status Please selectI am not a protect...,"<select aria-required=""false"" id=""job_applicat...",I am not a protected veteran,<NA>
1589,"Completing this form is voluntary, and we hope...","<a href=""https://www.dol.gov/ofccp"" target=""_b...",https://www.dol.gov/ofccp,<NA>


In [91]:
qe.loc[426,'label']='career level'
qe.loc[433,'label']='career level'
qe.loc[1305,'label']='career level'
qe.loc[1481,'label']='career level'

qe.loc[154,'label']='newsletter'
qe.loc[1480,'label']='newsletter'
qe.loc[1196,'label']='newsletter'
qe.loc[1530,'label']='sex'



qe.loc[1286,'label']='register'
qe.loc[320,'label']='login'

qe.loc[150,'label']='company'
qe.loc[1171,'label']='company'

qe.loc[320,'label']='login'
qe.loc[60,'label']='captcha'
qe.loc[63,'label']='captcha'
qe.loc[698,'label']='captcha'
qe.loc[734,'label']='captcha'



qe.loc[320,'label']='login'


qe.loc[1586,'label']='ethnicity'
qe.loc[81,'label']='skill'
qe.loc[1586,'label']='ethnicity'
qe.loc[1504,'label']='find us'
qe.loc[689,'label']='find us'
qe.loc[1532,'label']='sex'
qe.loc[1548,'label']='sex'

qe.loc[1291:1298,'label']='distance'
qe.loc[93,'label']='submit'
qe.loc[189,'label']='cancel'


qe.loc[500,'label']= pd.NA
qe.loc[501,'label']= pd.NA
qe.loc[146,'label']= pd.NA
qe.loc[295,'label']= pd.NA











In [88]:
from sentence_transformers.similarity_functions import cos_sim 
from IPython.display import clear_output

with torch.no_grad():
    for i,x in rows.iterrows():
        _q = embeding_model.encode(x['query'].lower())
        _l = embeding_model.encode(x['label'])
        sim = cos_sim(_q,_l)
        c = sim<0.4
        if c==True:
            print(i)
            print(sim)
            print(x['query'])
            print(x['element'])
            print(x['label'])
            print('--------------------------------------------------')
            #input()
            #clear_output(True)

        #break


57
tensor([[0.1788]])
Optional: Lebenslauf
<input aria-controls="filepond--assistant-mx4iq00m4" aria-labelledby="filepond--drop-label-mx4iq00m4" id="filepond--browser-mx4iq00m4" name="uploader_file" type="file"/>
cv
--------------------------------------------------
58
tensor([[0.0377]])
Optional: Weitere Unterlagen
<input aria-controls="filepond--assistant-qn43flav5" aria-labelledby="filepond--drop-label-qn43flav5" id="filepond--browser-qn43flav5" multiple="" name="uploader_file" type="file"/>
cv
--------------------------------------------------
64
tensor([[0.1846]])
1. Lebenslauf hochladen
<input id="f" name="f" type="file"/>
cv
--------------------------------------------------
66
tensor([[0.1373]])
Jetzt Lebenslauf hochladen
<input disabled="" id="btn_submit_cv" type="button" value="Jetzt Lebenslauf hochladen"/>
cv
--------------------------------------------------
118
tensor([[0.1345]])
Anschreiben hochladen
<input aria-autocomplete="list" id="react-select-2-input" readonly="" ta

In [92]:
qe.loc[32,'label']='cookie'
qe.loc[267,'label']='longitude'
qe.loc[1038,'label']='privacy'
qe.loc[1008,'label']='forgot password'
qe.loc[1452,'label']='forgot password'
qe.loc[1437,'label']='forgot password'
qe.loc[1321,'label']='forgot password'
qe.loc[98,'label']='forgot password'
qe.loc[1319,'label']='username'


qe.loc[929,'label']='recommender'
qe.loc[464,'label']='captcha'
qe.loc[369,'label']='captcha'

qe.loc[873,'label']='username'
qe.loc[250,'label']='last name'
qe.loc[155,'label']='pronoun'
qe.loc[141,'label']='pronoun'

qe.loc[123,'label']='job title'
qe.loc[763,'label']='job title'
qe.loc[273,'label']='job title'
qe.loc[229,'label']='job title'


qe.loc[102,'label']='sex'
qe.loc[101,'label']='sex'

qe.loc[125,'label']='available from'
qe.loc[1329,'label']='available from'

qe.loc[135,'label']='graduation year'
qe.loc[812,'label']='graduation year'

qe.loc[533,'label']='years of experience'
qe.loc[1274,'label']='salary'
qe.loc[1325,'label']='salary'
qe.loc[1330,'label']='salary'
qe.loc[551,'label']='salary'

qe.loc[1178,'label']='cover letter'
qe.loc[1132,'label']='cover letter'
qe.loc[1057,'label']='cover letter'
qe.loc[1032,'label']='cover letter'
qe.loc[1030,'label']='cover letter'
qe.loc[1032,'label']='cover letter'
qe.loc[1032,'label']='cover letter'






qe.loc[1494,'label']='major'
qe.loc[1490,'label']='major'

qe.loc[810,'label']='major'

qe.loc[1577,'label']='zip'
qe.loc[889,'label']='zip'

qe.loc[544,'label']='house number'
qe.loc[426,'label']='career level'
qe.loc[101,'label']='sex'
qe.loc[1277,'label']='picture'
qe.loc[1124,'label']='cancel'
qe.loc[101,'label']='sex'











In [93]:
#qe.to_excel('qe.xlsx',index=0)

In [80]:
qe.loc[146,'element']

'<input aria-required="true" id="449208e0-ffe6-ec11-bb3c-0022489dcb19" name="449208e0-ffe6-ec11-bb3c-0022489dcb19" required="required" type="checkbox"/>'